In [1]:
import requests
import io
import os
import re
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 4000)
pd.set_option('display.max_columns', 3000)
pd.set_option('display.width', 1000)
pd.options.display.max_seq_items = 2000
pd.options.mode.chained_assignment = None

urls = ['a2d435b3/Leicester-City', 
        '47c64c55/Crystal-Palace', 
        '822bd0ba/Liverpool', 
        'b8fd03ef/Manchester-City', 
        'cff3d9bb/Chelsea', 
        '19538871/Manchester-United', 
        '8cec06e1/Wolverhampton-Wanderers', 
        'cd051869/Brentford', 
        '361ca564/Tottenham-Hotspur', 
        '18bb7c10/Arsenal', 
        '943e8050/Burnley', 
        'd3fd31cc/Everton', 
        'b2b47a98/Newcastle-United', 
        '33c895d4/Southampton', 
        'd07537b9/Brighton--Hove-Albion', 
        '7c21e445/West-Ham-United', 
        '2abfe087/Watford', 
        '5bfb9659/Leeds-United', 
        '8602292d/Aston-Villa', 
        '1c781004/Norwich-City']


season = '2021-2022'
PATH = 'PlayerData'+'_'+season
PATH_Calendar = 'Calendar'+'_'+season
GAMES = 3
# -----------------------------------------------
GAMEWEEK = 38 # <------ нужно параметризовать!!!!
# -----------------------------------------------
INDEX = list(range(GAMEWEEK))[-GAMES:] # интересующие недели
os.makedirs(f'./results/GW{GAMEWEEK}', exist_ok=True)

def preprocessing(df, PLAYER, TEAM, DATE):
    data = [{
        'Player': PLAYER, 
        'Team': TEAM,
        'Opponent': df.Opponent.tolist(),
        'Start': df.Start.tolist(),
        'Min': df.Min.sum(),
        'Gls': df['Performance|Gls'].sum(),
        'Ast': df.Ast.sum(),
        'Sh': df['Performance|Sh'].sum(),
        'SoT': df['Performance|SoT'].sum(),
        'npxG': df['Expected|npxG'].sum(),
        'Att 3rd': df['Touches|Att 3rd'].sum(),
        'Att Pen': df['Touches|Att Pen'].sum(),
        'm_date': DATE
    }]

    return pd.DataFrame(data)

In [2]:
TEAMS = ['Leicester-City', 'Crystal-Palace', 'Liverpool', 'Manchester-City', 'Chelsea', 
        'Manchester-United', 'Wolverhampton-Wanderers', 'Brentford', 'Tottenham-Hotspur', 'Arsenal', 
        'Burnley', 'Everton', 'Newcastle-United', 'Southampton', 'Brighton--Hove-Albion', 
        'West-Ham-United', 'Watford', 'Leeds-United', 'Aston-Villa', 'Norwich-City']

abbrev = ['ARS', 'AVL', 'BRE', 'BRI', 'BUR', 'CHE', 'CRY', 'EVE', 'LEE', 'LEI', 'LIV', 
 'MCI', 'MUN', 'NEW', 'NOR', 'SOU', 'TOT', 'WAT', 'WHU', 'WOL']

last_N_games = {}

for i, team in tqdm(enumerate(sorted(TEAMS)), total=len(TEAMS)):

    FOR = pd.read_excel(f'./{PATH_Calendar}/{team}_for.xlsx')
    last_N_games[abbrev[i]] = FOR.Opponent[-GAMES:].tolist()

In [3]:
# print(filename_PATH)
# os.path.getmtime(filename_PATH)

In [4]:
stats = pd.DataFrame()

for path, subdirs, files in os.walk(PATH):
    for name in files:
        
        filename_PATH = os.path.join(path, name)
        TEAM = filename_PATH.split('/')[1]
        PLAYER = filename_PATH.split('/')[2][:-4]
        DATE = os.path.getmtime(filename_PATH) # для удаления дубликатов (трансферы внутри лиги)
        #print(TEAM, PLAYER)
        df = pd.read_csv(filename_PATH)        
        df.dropna(axis=0, how='all', inplace=True)
        df.reset_index(drop=True, inplace=True)
        try:
            df = df[['Round', 'Opponent', 'Start', 'Min', 'Performance|Gls', 'Ast', 'Performance|Sh', 
                 'Performance|SoT', 'Expected|npxG', 'Touches|Att 3rd', 'Touches|Att Pen']]
        except KeyError:
            continue
        df = df.drop(df[df.Round == 'Round'].index) # <-- в профиле Муссы Сиссоко строка-дубликат названий колонок
        # преобразование численных значений в числа
        cols_num = ['Min', 'Performance|Gls', 'Ast', 'Performance|Sh', 'Performance|SoT', 'Expected|npxG', 
                    'Touches|Att 3rd', 'Touches|Att Pen']
        for i in cols_num:
            df[i] = pd.to_numeric(df[i])
            
        if df.shape[0] == 0:
            continue
        else:
            #print( df['Round'] )
            df['Round'] = df['Round'].apply(lambda x: int(x.split(' ')[1]) )
            df.set_index('Round', inplace=True)
            
            # df = df[df.index.isin(INDEX)] # фильтр -- относительно номер GW
            
            df = df.iloc[-GAMES:, :] # фильтр -- последние три игры (плевать на последовательность)
            
            result = preprocessing(df, PLAYER, TEAM, DATE)
            stats = pd.concat([stats, result])

In [5]:
abbrev = ['ARS', 'AVL', 'BRE', 'BRI', 'BUR', 'CHE', 'CRY', 'EVE', 'LEE', 'LEI', 'LIV', 
 'MCI', 'MUN', 'NEW', 'NOR', 'SOU', 'TOT', 'WAT', 'WHU', 'WOL']
teams = sorted(stats.Team.unique().tolist())

teams_dict = dict(zip(teams, abbrev))
stats['Team'] = stats['Team'].map(teams_dict)

In [6]:
stats[f'Apps_last_{GAMES}GWs'] = stats['Team'].map(last_N_games)

In [7]:
stats.head()

,Player,Team,Opponent,Start,Min,Gls,Ast,Sh,SoT,npxG,Att 3rd,Att Pen,m_date,Apps_last_3GWs
0,Kasey McAteer,LEI,[Newcastle Utd],[N],3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.642180e+09,"[Norwich City, Watford, Chelsea]"
0,Wesley Fofana,LEI,"[Everton, Watford, Chelsea]","[Y, Y, Y]",270.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,1.653121e+09,"[Norwich City, Watford, Chelsea]"
0,Ricardo Pereira,LEI,"[Burnley, Arsenal, Everton]","[Y, Y, Y]",195.0,0.0,0.0,3.0,1.0,0.1,32.0,3.0,1.650654e+09,"[Norwich City, Watford, Chelsea]"
0,Çağlar Söyüncü,LEI,"[Newcastle Utd, Aston Villa, Tottenham]","[Y, Y, Y]",270.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,1.651591e+09,"[Norwich City, Watford, Chelsea]"
0,Hamza Choudhury,LEI,"[Tottenham, Leeds United, Manchester Utd]","[Y, Y, N]",151.0,0.0,0.0,1.0,0.0,0.0,5.0,1.0,1.649425e+09,"[Norwich City, Watford, Chelsea]"


In [8]:
# убираем дубликаты из-за трансферов внутри лиги
stats = stats.sort_values('m_date', ascending=False).drop_duplicates('Player')
stats.drop('m_date', axis=1, inplace=True)

In [9]:
def relevant_games(col_1, col_2):
    return GAMES - len(list(set(col_2) - set(col_1)))

stats['NumRelevantGames'] = stats.apply(lambda x: relevant_games(x['Opponent'], x[f'Apps_last_{GAMES}GWs']), axis=1)
stats.drop(f'Apps_last_{GAMES}GWs', axis=1, inplace=True)

In [10]:
stats['npxG/Sh'] = stats['npxG'] / stats['Sh']
stats['npxG/Sh'] = stats['npxG/Sh'].apply(lambda x: round(x, 2))

stats['Att Pen/Min'] = stats['Att Pen'] / stats['Min']
stats['Att Pen/Min'] = stats['Att Pen/Min'].apply(lambda x: round(x, 2))

In [11]:
stats = stats.sort_values(by = 'Att Pen', ascending=False)

In [12]:
stats.head()

,Player,Team,Opponent,Start,Min,Gls,Ast,Sh,SoT,npxG,Att 3rd,Att Pen,NumRelevantGames,npxG/Sh,Att Pen/Min
0,Jack Grealish,MCI,"[Newcastle Utd, Wolves, West Ham]","[Y, N, Y]",190.0,1.0,1.0,7.0,1.0,0.4,98.0,24.0,3,0.06,0.13
0,Leandro Trossard,BRI,"[Wolves, Manchester Utd, Leeds United]","[Y, Y, Y]",256.0,2.0,2.0,6.0,3.0,1.2,85.0,24.0,3,0.20,0.09
0,Raheem Sterling,MCI,"[Leeds United, Newcastle Utd, Wolves]","[Y, Y, Y]",270.0,3.0,1.0,8.0,3.0,3.3,92.0,24.0,2,0.41,0.09
0,Romelu Lukaku,CHE,"[Wolves, Leeds United, Leicester City]","[Y, Y, Y]",256.0,3.0,0.0,11.0,3.0,1.3,66.0,24.0,3,0.12,0.09
0,Richarlison,EVE,"[Watford, Brentford, Crystal Palace]","[Y, Y, Y]",269.0,2.0,1.0,10.0,3.0,0.9,74.0,23.0,3,0.09,0.09


In [13]:
# stats.to_excel(f'./results/GW{GAMEWEEK}/offensive_stats_preGW{GAMEWEEK}_last_{GAMES}GWs.xlsx', index=False)

In [14]:
# saving for last GAMES played independant on consistency with information on last games for a team
stats.to_excel(f'./results/GW{GAMEWEEK}/offensive_stats_preGW{GAMEWEEK}_last_{GAMES}_for_a_player.xlsx', index=False)

---
---
---

In [17]:
cond1 = stats.NumRelevantGames.isin([2, 3])
#cond2 = stats.Team.isin(['ARS', 'WOL', 'LEE', 'AVL'])

stats[cond1].sort_values(['Att Pen', 'npxG/Sh'], ascending=[False, False])

,Player,Team,Opponent,Start,Min,Gls,Ast,Sh,SoT,npxG,Att 3rd,Att Pen,NumRelevantGames,npxG/Sh,Att Pen/Min
0,Ollie Watkins,AVL,"[Tottenham, Leicester City, Norwich City]","[Y, Y, Y]",270.0,1.0,0.0,12.0,4.0,1.2,71.0,27.0,3,0.10,0.10
0,Mohamed Salah,LIV,"[Manchester Utd, Everton, Newcastle Utd]","[Y, Y, N]",202.0,2.0,2.0,7.0,3.0,0.9,107.0,25.0,3,0.13,0.12
0,Mason Mount,CHE,"[West Ham, Manchester Utd, Everton]","[Y, Y, Y]",270.0,0.0,0.0,11.0,0.0,0.6,127.0,23.0,3,0.05,0.09
0,Ivan Toney,BRE,"[Watford, Tottenham, Manchester Utd]","[Y, Y, Y]",270.0,0.0,0.0,10.0,1.0,0.9,67.0,22.0,3,0.09,0.08
0,Kai Havertz,CHE,"[West Ham, Manchester Utd, Everton]","[Y, Y, Y]",234.0,0.0,1.0,7.0,2.0,1.1,91.0,21.0,3,0.16,0.09
0,Raheem Sterling,MCI,"[Liverpool, Watford, Leeds United]","[Y, Y, Y]",254.0,0.0,0.0,6.0,1.0,1.5,64.0,20.0,2,0.25,0.08
0,Luis Díaz,LIV,"[Manchester Utd, Everton, Newcastle Utd]","[Y, N, Y]",190.0,1.0,2.0,10.0,5.0,1.0,64.0,20.0,3,0.10,0.11
0,Gabriel Jesus,MCI,"[Brighton, Watford, Leeds United]","[N, Y, Y]",188.0,5.0,1.0,7.0,5.0,1.7,76.0,19.0,3,0.24,0.10
0,Eddie Nketiah,ARS,"[Chelsea, Manchester Utd, West Ham]","[Y, Y, Y]",249.0,2.0,0.0,12.0,7.0,2.1,52.0,19.0,3,0.18,0.08
0,Bryan Mbeumo,BRE,"[Watford, Tottenham, Manchester Utd]","[Y, Y, Y]",255.0,0.0,0.0,10.0,1.0,0.7,62.0,19.0,3,0.07,0.07


---

In [23]:
TEAM = 'WAT'
PLAYER = 'Moussa Sissoko'
df = pd.read_csv('./'+PATH+'/Watford/Moussa Sissoko.csv')
df = df[['Round', 'Opponent', 'Start', 'Min', 'Performance|Gls', 'Ast', 'Performance|Sh', 
                 'Performance|SoT', 'Expected|npxG', 'Touches|Att 3rd', 'Touches|Att Pen']]
df = df.drop(df[df.Round == 'Round'].index)
cols_num = ['Min', 'Performance|Gls', 'Ast', 'Performance|Sh', 
            'Performance|SoT', 'Expected|npxG', 'Touches|Att 3rd', 'Touches|Att Pen']

for i in cols_num:
    df[i] = pd.to_numeric(df[i])

df

,Round,Opponent,Start,Min,Performance|Gls,Ast,Performance|Sh,Performance|SoT,Expected|npxG,Touches|Att 3rd,Touches|Att Pen
1,Matchweek 3,Tottenham,Y,70,0,0,1,0,0.1,6,2
2,Matchweek 4,Wolves,Y,90,0,0,1,1,0.1,15,2
3,Matchweek 5,Norwich City,Y,90,0,0,0,0,0.0,5,0
4,Matchweek 6,Newcastle Utd,Y,90,0,0,1,1,0.0,13,0
5,Matchweek 7,Leeds United,Y,90,0,0,0,0,0.0,5,0
6,Matchweek 8,Liverpool,Y*,90,0,0,0,0,0.0,4,0
7,Matchweek 9,Everton,Y*,90,0,0,1,0,0.0,8,0
8,Matchweek 10,Southampton,Y*,90,0,0,0,0,0.0,5,0
9,Matchweek 11,Arsenal,Y*,90,0,0,0,0,0.0,8,1
10,Matchweek 12,Manchester Utd,Y*,90,0,0,2,0,0.1,12,2


In [24]:
df['Round'] = df['Round'].apply(lambda x: int(x.split(' ')[1]) )
df.set_index('Round', inplace=True)
df

,Opponent,Start,Min,Performance|Gls,Ast,Performance|Sh,Performance|SoT,Expected|npxG,Touches|Att 3rd,Touches|Att Pen
Round,,,,,,,,,,
3,Tottenham,Y,70,0,0,1,0,0.1,6,2
4,Wolves,Y,90,0,0,1,1,0.1,15,2
5,Norwich City,Y,90,0,0,0,0,0.0,5,0
6,Newcastle Utd,Y,90,0,0,1,1,0.0,13,0
7,Leeds United,Y,90,0,0,0,0,0.0,5,0
8,Liverpool,Y*,90,0,0,0,0,0.0,4,0
9,Everton,Y*,90,0,0,1,0,0.0,8,0
10,Southampton,Y*,90,0,0,0,0,0.0,5,0
11,Arsenal,Y*,90,0,0,0,0,0.0,8,1


In [28]:
df.iloc[-3:, :]

,Opponent,Start,Min,Performance|Gls,Ast,Performance|Sh,Performance|SoT,Expected|npxG,Touches|Att 3rd,Touches|Att Pen
Round,,,,,,,,,,
14,Chelsea,Y*,90,0,1,1,1,0.3,21,3
15,Manchester City,Y*,90,0,0,2,0,0.1,6,0
16,Brentford,Y*,90,0,0,0,0,0.0,3,0


In [17]:
INDEX

[14, 15, 16, 17, 18]

In [16]:
df = df[df.index.isin(INDEX)]
df

,Opponent,Start,Min,Performance|Gls,Ast,Performance|Sh,Performance|SoT,Expected|npxG,Touches|Att 3rd,Touches|Att Pen
Round,,,,,,,,,,
14,Chelsea,Y*,90,0,1,1,1,0.3,21,3
15,Manchester City,Y*,90,0,0,2,0,0.1,6,0
16,Brentford,Y*,90,0,0,0,0,0.0,3,0


In [26]:
preprocessing(df, PLAYER, TEAM)

,Player,Team,Opponent,Start,Min,Gls,Ast,Sh,SoT,npxG,Att 3rd,Att Pen
0,Moussa Sissoko,LEI,"[Everton, Southampton, Arsenal]","[Y*, Y*, Y*]",270,0,0,1,0,0.0,21,1
